In [ ]:
import time
import pandas as pd
import numpy as np
import datetime
import warnings
warnings.filterwarnings("ignore")
import re
from glob import glob
from tqdm import tqdm
from wordcloud import WordCloud 
# get_ipython().run_line_magic('matplotlib', 'inline')
from matplotlib import font_manager, rc
import platform
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from konlpy.tag import Okt
from collections import Counter
import missingno

# 데이터 로드 및 병합

여러가지 자연어 감성 라벨링 데이터를 불러와 동일한 형태의 데이터 프레임으로 변환해준 뒤 하나의 데이터프레임으로 병합

부정 = 0
긍정 = 1
중립 = 2

으로 감성 라벨링 된 열로 통일

In [ ]:
# 한글 폰트 위치를 넣어주세요 (위에서 나눔고딕 파일을 해당 경로에 복사붙여넣기 했을 경우 그대로 실행하셔도 무방합니다.)
font_name = matplotlib.font_manager.FontProperties(fname="C:/Windows/Fonts/malgun.ttf").get_name() # NanumGothic.otf
matplotlib.rc('font', family=font_name)

In [ ]:
# 영화 리뷰 불러오기 - 긍/부정 = 1/0
movie_df = pd.read_csv('./ratings.txt',sep = "\t", engine='python')
# 네이버 쇼핑 리뷰 불러오기 - rating(0~5의 별점으로 표기)
shopping_df = pd.read_table('naver_shopping.txt', names=['ratings', 'reviews'])
# Steam 게임 리뷰 불러오기 - 긍/부정 = 1/0
steam_df = pd.read_table('steam.txt', names=['label', 'reviews'])
# # 트위터 영어 번역본 불러오기
# twitter_df_test = pd.read_excel('./test번역.xlsx')
# twitter_df_train = pd.read_excel('./training번역.xlsx')
# twitter_df = pd.concat([twitter_df_test,twitter_df_train],axis=0)
# 일반 대화 데이터셋 - 7개의 감정 (중립, 놀람, 분노, 슬픔, 행복, 혐오, 공포)
sin_dialog_df = pd.read_excel('./한국어_단발성_대화_데이터셋.xlsx',usecols=[0, 1])
con_dialog_df = pd.read_excel('./한국어_연속적_대화_데이터셋.xlsx',usecols=[0, 1])
dialog_df = pd.concat([sin_dialog_df,con_dialog_df],axis=0)
# 욕설 데이터 셋 - 욕설 포함시 레이블 1 
curse_df = pd.read_csv('./curse.txt',sep = "|",names=['content', 'label'])
# curse_df 잘못된 행 삭제 및 label을 int 타입으로 변경
curse_df = curse_df.drop([455])
curse_df['label']= curse_df['label'].astype(int)
# 챗봇 데이터 셋 - 일상다반사 0, 이별(부정) 1, 사랑(긍정) 2로 레이블링
chatbot_df = pd.read_csv('./ChatbotData.txt',sep=',',usecols=[0,2])

In [ ]:
def emotion_to_label(emotion):
    if emotion == '중립' or emotion == '놀람':
        return 2
    elif emotion =='행복':
        return 1
    else:
        return 0
    
def rating_to_label(rating):
    if rating ==3:
        return 2
    elif rating >3:
        return 1
    else:
        return 0
    
def curse_to_label(curse):
    if curse == 1:
        return 0
    else:
        return 2

def chatbot_to_label(label):
    if label== 0:
        return 2
    elif label ==1:
        return 0
    else: 
        return 1

In [ ]:
# 감정을 부정/긍정/중립 = 0/1/2 로 구분
dialog_df['label'] = dialog_df['Emotion'].apply(emotion_to_label)
# 평점을 부정/긍정/중립 = 0/1/2 로 구분
shopping_df['label'] = shopping_df['ratings'].apply(rating_to_label)
# 욕설 포함시 부정, 미포함시 중립 으로 변경
curse_df['label'] = curse_df['label'].apply(curse_to_label)
# 사랑(긍정) 은 긍정(1), 이벌(부정) 은 부정(0), 일상다반사는 중립(2) 으로 변경
chatbot_df['label'] = chatbot_df['label'].apply(chatbot_to_label)


In [ ]:
# movie_df, shopping_df, dialog_df는 label 열과 문장열만 남긴다
movie_df= movie_df.drop('id',axis=1)
shopping_df =shopping_df.drop('ratings',axis=1)
dialog_df= dialog_df.drop('Emotion',axis=1)
# steam df는 열 순서를 변경해준다
steam_df= steam_df[['reviews','label']]


In [ ]:
# 컬럼명 통일해주기
movie_df.columns =['sentence','label']
shopping_df.columns =['sentence','label']
dialog_df.columns =['sentence','label']
steam_df.columns =['sentence','label']
curse_df.columns =['sentence','label']
chatbot_df.columns =['sentence','label']

In [ ]:
data_df = pd.concat([movie_df,shopping_df,dialog_df,steam_df,curse_df,chatbot_df],ignore_index=True,)

In [ ]:
# 결측치 제거 및 인덱스 재정렬
data_df = data_df.dropna().reset_index().drop('index',axis=1)
# 중복행 제거
data_df = data_df.drop_duplicates(['sentence'])

In [ ]:
data_df

,sentence,label
0,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1
...,...,...
611854,후회 없이 사랑하고 싶어,1
611855,훔쳐보는 것도 눈치 보임.,1
611857,흑기사 해주는 짝남.,1
611858,힘든 연애 좋은 연애라는게 무슨 차이일까?,1


In [ ]:
# 부정/긍정/중립 개수
data_df['label'].value_counts()

0    279932
1    256875
2     64172
Name: label, dtype: int64

In [ ]:
data_df.to_csv('./data_df.csv')